# ENCODAGES ET SÉPARATION TRAIN/TEST 

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
## paths
folder_path_M = '/Users/maximehenon/Documents/GitHub/MAR25_BDS_Compagnon_Immo/'
# folder_path_Y = 'C:/Users/charl/OneDrive/Documents/Yasmine/DATASCIENTEST/FEV25-BDS-COMPAGNON'
#folder_path_C = '../data/processed/Sales'
# folder_path_L= '/Users/loick.d/Documents/Datascientest/Github immo/MAR25_BDS_Compagnon_Immo/'
# folder_path_LW = 'C:/Users/User/Downloads/drive-download-20250508T155351Z-1-001'


# Load the dataset
output_file = os.path.join(folder_path_M, 'df_sales_clean.csv')
# output_file = os.path.join(folder_path_Y, 'df_sales_clean.csv')
#output_file = os.path.join(folder_path_C, 'df_sales_clean.csv')
#output_file = os.path.join(folder_path_L, 'df_sales_clean.csv')
# output_file = os.path.join(folder_path_LW, 'df_sales_clean.csv')

chunksize = 100000  # Number of rows per chunk
chunks = pd.read_csv(output_file, sep=';', chunksize=chunksize, index_col=None, on_bad_lines='skip', low_memory=False)
# Process chunks
df_sales_clean = pd.concat(chunk for chunk in chunks)

## Rappel des colonnes restantes
# print("Colonnes restantes dans le DataFrame :")
# print(df_sales_clean.columns)
# print(df_sales_clean.dtypes)
# print("\nShape du Dataset après élimination des colonnes :", df_sales_clean.shape)
print(df_sales_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4873455 entries, 0 to 4873454
Data columns (total 37 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   typedebien                   object 
 1   typedetransaction            object 
 2   etage                        int64  
 3   surface                      int64  
 4   surface_terrain              float64
 5   nb_pieces                    int64  
 6   balcon                       int64  
 7   eau                          int64  
 8   bain                         int64  
 9   dpeL                         object 
 10  dpeC                         float64
 11  mapCoordonneesLatitude       float64
 12  mapCoordonneesLongitude      float64
 13  nb_etages                    float64
 14  places_parking               float64
 15  cave                         bool   
 16  exposition                   object 
 17  ges_class                    object 
 18  annee_construction           object 
 19  

## Encodages


### Catégorielles ordinales

In [ ]:
# Categorielles Ordinales    #
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pandas as pd




# Liste des colonnes ordinales
ordinales = ["ges_class", "dpeL", "logement_neuf", "nb_pieces", "bain", "eau", "nb_toilettes", "balcon"]

# Création d'une pipeline pour l'imputation et l'encodage
pipeline_ordinale = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("encode", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

# Application de la pipeline sur les colonnes ordinales
ord_transformed = pipeline_ordinale.fit_transform(df_sales_clean[ordinales])

# Conversion du résultat en DataFrame
ord_columns = ordinales
ord_encoded = pd.DataFrame(ord_transformed, columns=ord_columns, index=df_sales_clean.index)

# Remplacement des colonnes originales par les colonnes encodées
df_sales_clean = df_sales_clean.drop(columns=ordinales).join(ord_encoded)

# Affichage des colonnes encodées
print("Colonnes encodées :")
print(ord_encoded.columns)
print("\nDtypes des colonnes encodées :")
print(ord_encoded.dtypes)
print("\nShape du DataFrame après encodage des colonnes ordinales :", df_sales_clean.shape)

Colonnes encodées :
Index(['ges_class', 'dpeL', 'logement_neuf', 'nb_pieces', 'bain', 'eau',
       'nb_toilettes', 'balcon'],
      dtype='object')

Dtypes des colonnes encodées :
ges_class        float64
dpeL             float64
logement_neuf    float64
nb_pieces        float64
bain             float64
eau              float64
nb_toilettes     float64
balcon           float64
dtype: object

Shape du DataFrame après encodage des colonnes ordinales : (4873455, 37)


### Catégorielles non-ordinales

In [4]:
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

# colonne de regroupement
GROUP_COL    = 'INSEE_COM'  

# Identification des colonnes pseudo-catégoriques
pseudo_categorical_ints = [col for col in ['etage', 'nb_etages'] if col in df_sales_clean.columns] 
# Conversion en string
df_sales_clean[pseudo_categorical_ints] = df_sales_clean[pseudo_categorical_ints].astype(str)


# ---------- Étape 1 : Gestion des valeurs manquantes ----------
# (Imputation des colonnes pseudo-catégoriques et textuelles)

# Identifier les colonnes avec des valeurs manquantes
missing_cols = df_sales_clean.columns[df_sales_clean.isnull().any()]
print("Colonnes avec des valeurs manquantes :")
print(missing_cols)

# Colonnes pseudo-catégoriques avec des valeurs manquantes
missing_pseudo_categorical = [col for col in pseudo_categorical_ints if col in missing_cols]
print("Colonnes pseudo-catégoriques avec des valeurs manquantes :")
print(missing_pseudo_categorical)

# Colonnes textuelles avec des valeurs manquantes
missing_text_cols = [col for col in df_sales_clean.select_dtypes(include=['object']).columns if col in missing_cols]
print("Colonnes textuelles avec des valeurs manquantes :")
print(missing_text_cols)

# Imputation des colonnes pseudo-catégoriques par la médiane par INSEE_COM
for col in missing_pseudo_categorical:
    group_medians = df_sales_clean.groupby(GROUP_COL)[col].median()
    global_median = df_sales_clean[col].median()
    df_sales_clean[col] = df_sales_clean[GROUP_COL].map(group_medians).fillna(global_median)

# Imputation des colonnes textuelles avec une valeur par défaut
for col in missing_text_cols:
    df_sales_clean[col] = df_sales_clean[col].fillna("MISSING")

# Vérification
print("Valeurs manquantes après imputation :")
print(df_sales_clean[missing_cols].isnull().sum())


# ---------- Étape 2: lister les colonnes catégoriques ----------

# Conversion des colonnes pseudo-catégoriques en string pour les traiter comme catégoriques
df_sales_clean[pseudo_categorical_ints] = df_sales_clean[pseudo_categorical_ints].astype(str)
cat_cols = [col for col in pseudo_categorical_ints]

# Ajout des colonnes de type object
object_cols = df_sales_clean.select_dtypes(include=['object']).columns
cat_cols.extend([col for col in object_cols if col not in cat_cols])

# Exclusions
cat_cols = [col for col in cat_cols if col not in ordinales]
cat_cols = [col for col in cat_cols if df_sales_clean[col].dtype != 'bool']
cat_cols = [col for col in cat_cols if col != GROUP_COL]
cat_cols = [col for col in cat_cols if col in df_sales_clean.columns]


# ---------- Etape 2: séparer les colonnes catégoriques en fonction de leur cardinalité ----------
cat_cols_large = [col for col in cat_cols if df_sales_clean[col].nunique() > 10]
cat_cols_small = [col for col in cat_cols if df_sales_clean[col].nunique() <= 10]


# ---------- Etape 3: Appliquer un Target-Encoding pour les colonnes à forte cardinalité  ----------

if cat_cols_large:
    df_sales_clean[cat_cols_large] = df_sales_clean[cat_cols_large].fillna("MISSING")
    target_encoder = TargetEncoder(cols=cat_cols_large)
    encoded_large = target_encoder.fit_transform(df_sales_clean[cat_cols_large], df_sales_clean["prix_m2_vente"])
    df_sales_clean.drop(columns=cat_cols_large, inplace=True)
    df_sales_clean = df_sales_clean.join(
        pd.DataFrame(encoded_large, columns=cat_cols_large, index=df_sales_clean.index))
else:
    print("Aucune colonne à plus de 10 modalités.")


# ---------- Etape 4: Appliquer un One_hot encoding pour les coloones à faible cardinalité ----------
if cat_cols_small:
    one_hot_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value="MISSING")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])
    ohe_encoded = one_hot_pipeline.fit_transform(df_sales_clean[cat_cols_small])
    ohe_col_names = one_hot_pipeline.named_steps["onehot"].get_feature_names_out(cat_cols_small)
    df_sales_clean.drop(columns=cat_cols_small, inplace=True)
    df_sales_clean = df_sales_clean.join(
        pd.DataFrame(ohe_encoded, columns=ohe_col_names, index=df_sales_clean.index))
else:
    print("Aucune colonne à 10 modalités ou moins.")

# ---------- Etape 5: Vérification des colonnes encodées ----------
# Affichage des colonnes encodées en target et en one-hot
print("Colonnes encodées en target :")
print(cat_cols_large)
print("\nColonnes encodées en one-hot :\n")
print(cat_cols_small)

# ----------- Etape 6 : supprimer les colonnes initiales après encodage ----------

df_sales_clean.drop(columns=[col for col in cat_cols_small if col in df_sales_clean.columns], inplace=True)

# Vérification après suppression
print("\nColonnes One-Hot initiales supprimées :", [col for col in cat_cols_small if col not in df_sales_clean.columns])


Colonnes avec des valeurs manquantes :
Index(['surface_terrain', 'dpeC', 'places_parking', 'exposition',
       'annee_construction', 'charges_copro', 'chauffage_energie',
       'chauffage_systeme', 'chauffage_mode', 'loyer_m2_median_n6',
       'nb_log_n6', 'taux_rendement_n6', 'loyer_m2_median_n7', 'nb_log_n7',
       'taux_rendement_n7', 'chauffage_energie_principal'],
      dtype='object')
Colonnes pseudo-catégoriques avec des valeurs manquantes :
[]
Colonnes textuelles avec des valeurs manquantes :
['exposition', 'annee_construction', 'chauffage_energie', 'chauffage_systeme', 'chauffage_mode', 'chauffage_energie_principal']
Valeurs manquantes après imputation :
surface_terrain                2439719
dpeC                           1692044
places_parking                 3073570
exposition                           0
annee_construction                   0
charges_copro                  3562782
chauffage_energie                    0
chauffage_systeme                    0
chauffage_mo

### Variables géographiques

In [5]:
import numpy as np

# Conversion des coordonnées géographiques en radians
lat_rad = np.radians(df_sales_clean['mapCoordonneesLatitude'].values)
lon_rad = np.radians(df_sales_clean['mapCoordonneesLongitude'].values)

# Projection sur la sphère unité
df_sales_clean['x_geo'] = np.cos(lat_rad) * np.cos(lon_rad)
df_sales_clean['y_geo'] = np.cos(lat_rad) * np.sin(lon_rad)
df_sales_clean['z_geo'] = np.sin(lat_rad)

# Suppression des colonnes Latitude et Longitude
df_sales_clean = df_sales_clean.drop(columns=['mapCoordonneesLongitude', 'mapCoordonneesLatitude'])

# Affichage des colonnes encodées
print("Colonnes géographiques :")
print(['x_geo', 'y_geo', 'z_geo'])
print("\nDtypes des colonnes géographiques :")
print(df_sales_clean[['x_geo', 'y_geo', 'z_geo']].dtypes)

Colonnes géographiques :
['x_geo', 'y_geo', 'z_geo']

Dtypes des colonnes géographiques :
x_geo    float64
y_geo    float64
z_geo    float64
dtype: object


### Variables numériques continues

In [6]:
import numpy as np

GROUP_COL = 'INSEE_COM'  # Colonne de regroupement

# ---------- Étape 1 : Imputation des valeurs manquantes pour les colonnes numériques ----------

# Sélection des colonnes numériques
numeric_cols = df_sales_clean.select_dtypes(include=['int64', 'float64']).columns

# Exclusion des colonnes ordinales, pseudo-catégoriques, catégoriques (small et large), géographiques, et One-Hot encodées
numeric_cols = [
    col for col in numeric_cols
    if col not in (ordinales + pseudo_categorical_ints + cat_cols_small + cat_cols_large + list(ohe_col_names) + ['x_geo', 'y_geo', 'z_geo'])
]

# Imputation des valeurs manquantes par la médiane par groupe (INSEE_COM)
for col in numeric_cols:
    # Calcul de la médiane par groupe
    group_medians = df_sales_clean.groupby(GROUP_COL)[col].median()

    # Remplacement des valeurs manquantes par la médiane du groupe, sinon par la médiane globale
    group_values = df_sales_clean[GROUP_COL].map(group_medians)
    global_median = df_sales_clean[col].median()
    df_sales_clean[col] = df_sales_clean[col].fillna(group_values).fillna(global_median)

# Colonnes traitées
print("Colonnes numériques traitées :")
print(numeric_cols)
print("\nValeurs manquantes après imputation :")
print(df_sales_clean[numeric_cols].isnull().sum())

Colonnes numériques traitées :
['surface', 'surface_terrain', 'dpeC', 'places_parking', 'charges_copro', 'loyer_m2_median_n6', 'nb_log_n6', 'taux_rendement_n6', 'loyer_m2_median_n7', 'nb_log_n7', 'taux_rendement_n7', 'prix_m2_vente']

Valeurs manquantes après imputation :
surface               0
surface_terrain       0
dpeC                  0
places_parking        0
charges_copro         0
loyer_m2_median_n6    0
nb_log_n6             0
taux_rendement_n6     0
loyer_m2_median_n7    0
nb_log_n7             0
taux_rendement_n7     0
prix_m2_vente         0
dtype: int64


## Standardisations

In [7]:
from sklearn.preprocessing import StandardScaler

# Initialisation du scaler
scaler = StandardScaler()

# ---------- Étape 1 : Standardisation des variables numériques continues ----------


df_sales_clean[numeric_cols] = scaler.fit_transform(df_sales_clean[numeric_cols])

# ---------- Étape 2 : Standardisation des variables catégorielles ordinales ----------

# Standardisation des colonnes ordinales
df_sales_clean[ordinales] = scaler.fit_transform(df_sales_clean[ordinales])

# ---------- Étape 3 : Standardisation des variables encodées avec TargetEncoder ----------

# Standardisation des colonnes encodées avec TargetEncoder
df_sales_clean[cat_cols_large] = scaler.fit_transform(df_sales_clean[cat_cols_large])


### Vérifications

In [8]:

# Vérifications
print("Colonnes standardisées :")
print("    Numériques continues :\n","       ", numeric_cols)
print("    Ordinales :\n", "       ",ordinales)
print("    Encodées avec TargetEncoder :\n", "       ",cat_cols_large)

# Liste des colonnes non standardisées par soustraction
standardized_cols = numeric_cols + ordinales + cat_cols_large
non_standardized_cols = [col for col in df_sales_clean.columns if col not in standardized_cols]

# Affichage des colonnes non standardisées
print("\n\n Colonnes non standardisées après vérification :")
print("     ", non_standardized_cols)
print("\n\n")
# Verification des colonnes restantes avant le split
print("Colonnes restantes dans df_sales_clean après les encodages:\n")
print("     ",df_sales_clean.columns)

Colonnes standardisées :
    Numériques continues :
         ['surface', 'surface_terrain', 'dpeC', 'places_parking', 'charges_copro', 'loyer_m2_median_n6', 'nb_log_n6', 'taux_rendement_n6', 'loyer_m2_median_n7', 'nb_log_n7', 'taux_rendement_n7', 'prix_m2_vente']
    Ordinales :
         ['ges_class', 'dpeL', 'logement_neuf', 'nb_pieces', 'bain', 'eau', 'nb_toilettes', 'balcon']
    Encodées avec TargetEncoder :
         ['etage', 'nb_etages', 'exposition', 'annee_construction', 'chauffage_energie', 'chauffage_systeme', 'date']


 Colonnes non standardisées après vérification :
      ['cave', 'porte_digicode', 'ascenseur', 'INSEE_COM', 'typedebien_Maison/Villa neuve', 'typedebien_a', 'typedebien_an', 'typedebien_h', 'typedebien_l', 'typedebien_m', 'typedebien_mn', 'typedetransaction_pi', 'typedetransaction_v', 'typedetransaction_vp', 'chauffage_mode_Central', 'chauffage_mode_Collectif', 'chauffage_mode_Collectif, Central', 'chauffage_mode_Collectif, Individuel', 'chauffage_mode_Collect

## Split

In [9]:
#SPLIT 
from sklearn.model_selection import train_test_split

X = df_sales_clean.drop(columns=['prix_m2_vente', 'INSEE_COM'])
y = df_sales_clean['prix_m2_vente']

# Séparation du jeu de données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Afficher les shapes des datasets
print("Shape de X_train :", X_train.shape)
print("Shape de X_test :", X_test.shape)
print("Shape de y_train :", y_train.shape)
print("Shape de y_test :", y_test.shape)



Shape de X_train : (3898764, 55)
Shape de X_test : (974691, 55)
Shape de y_train : (3898764,)
Shape de y_test : (974691,)


### Sauvegarde

In [10]:
## Paths
folder_path_M = '/Users/maximehenon/Documents/GitHub/MAR25_BDS_Compagnon_Immo/'
# folder_path_Y = 'C:/Users/charl/OneDrive/Documents/Yasmine/DATASCIENTEST/FEV25-BDS-COMPAGNON'
#folder_path_C = '../data/processed/Sales'
#folder_path_L = '/Users/loick.d/Documents/Datascientest/Github immo/MAR25_BDS_Compagnon_Immo/'
# folder_path_LW = 'C:/Users/User/Downloads/drive-download-20250508T155351Z-1-001'

## stocker les datasets
X_test.to_csv(os.path.join(folder_path_M, 'X_test_encoded.csv'), sep=';', index=False)
X_train.to_csv(os.path.join(folder_path_M, 'X_train_encoded.csv'), sep=';', index=False)
y_train.to_csv(os.path.join(folder_path_M, 'y_train_encoded.csv'), sep=';', index=False)
y_test.to_csv(os.path.join(folder_path_M, 'y_test_encoded.csv'), sep=';', index=False)

# X_test.to_csv(os.path.join(folder_path_Y, 'X_test_encoded.csv'), sep=';', index=False)
# X_train.to_csv(os.path.join(folder_path_Y, 'X_train_encoded.csv'), sep=';', index=False)
# y_train.to_csv(os.path.join(folder_path_Y, 'y_train_encoded.csv'), sep=';', index=False)
# y_test.to_csv(os.path.join(folder_path_Y, 'y_test_encoded.csv'), sep=';', index=False)

# X_test.to_csv(os.path.join(folder_path_C, 'X_test_encoded.csv'), sep=';', index=False)
# X_train.to_csv(os.path.join(folder_path_C, 'X_train_encoded.csv'), sep=';', index=False)
# y_train.to_csv(os.path.join(folder_path_C, 'y_train_encoded.csv'), sep=';', index=False)
# y_test.to_csv(os.path.join(folder_path_C, 'y_test_encoded.csv'), sep=';', index=False)

# X_test.to_Lsv(os.path.join(folder_path_L, 'X_test_encoded.csv'), sep=';', index=False)
# X_train.to_Lsv(os.path.join(folder_path_L, 'X_train_encoded.csv'), sep=';', index=False)
# y_train.to_Lsv(os.path.join(folder_path_L, 'y_train_encoded.csv'), sep=';', index=False)
# y_test.to_Lsv(os.path.join(folder_path_L, 'y_test_encoded.csv'), sep=';', index=False)

# X_test.to_csv(os.path.join(folder_path_LW, 'X_test_encoded.csv'), sep=';', index=False)
# X_train.to_csv(os.path.join(folder_path_LW, 'X_train_encoded.csv'), sep=';', index=False)
# y_train.to_csv(os.path.join(folder_path_LW, 'y_train_encoded.csv'), sep=';', index=False)
# y_test.to_csv(os.path.join(folder_path_LW, 'y_test_encoded.csv'), sep=';', index=False)

